In [1]:
print(4)

4


In [21]:
#import
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
#from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import PowerTransformer
#all metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
#random forest
from sklearn.ensemble import RandomForestClassifier
# import StratifiedKFold
from sklearn.model_selection import StratifiedKFold



def map_gambling(df):
    df['CAT_GAMBLING'] = df['CAT_GAMBLING'].map({'No': 0, 'Low': 1,'High': 2})
    return df

def split_X_from_y(df, target='DEFAULT'):
    X = df.drop(columns=[target])
    y = df[target]
    return X, y

def get_my_metrics(model, X_train, X_val, y_train, y_val):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1= f1_score(y_val, y_pred)
    roc_auc = roc_auc_score(y_val, y_pred)
    
    return accuracy, precision, recall, f1, roc_auc

def cox_box_transform(X_train, X_val):
    #transform data
    #scaler = StandardScaler()
    cox_box_model = PowerTransformer(method='yeo-johnson')
    X_train_transformed=cox_box_model.fit_transform(X_train)
    X_val_transformed = cox_box_model.transform(X_val)
    
    #X_train_transformed = scaler.fit_transform(X_train_transformed)
    #X_val_transformed = scaler.transform(X_val_transformed)
    
    return X_train_transformed, X_val_transformed


def create_tally(X_train, X_val, y_train, y_val, models, names):
    
     
    X_train_transformed, X_val_transformed = cox_box_transform(X_train, X_val)
    model.fit(X_train_transformed, y_train)
    y_pred = model.predict(X_val_transformed)
    
    f1= f1_score(y_val, y_pred)
    
    
    
    return f1


In [22]:
#load data
df_train=pd.read_csv('../data/for_modelling/credit_score_train.csv')
df_val=pd.read_csv('../data/for_modelling/credit_score_valid.csv')

#map gambling
df_train = map_gambling(df_train)
df_val = map_gambling(df_val)

X_train, y_train = split_X_from_y(df_train)
X_val, y_val = split_X_from_y(df_val)

X_train


,INCOME,SAVINGS,DEBT,R_SAVINGS_INCOME,R_DEBT_INCOME,R_DEBT_SAVINGS,T_CLOTHING_12,T_CLOTHING_6,R_CLOTHING,R_CLOTHING_INCOME,...,R_EXPENDITURE_INCOME,R_EXPENDITURE_SAVINGS,R_EXPENDITURE_DEBT,CAT_GAMBLING,CAT_DEBT,CAT_CREDIT_CARD,CAT_MORTGAGE,CAT_SAVINGS_ACCOUNT,CAT_DEPENDENTS,CREDIT_SCORE
0,2783,1855,0,0.6665,0.0000,0.0000,103,74,0.7184,0.0370,...,1.6665,2.5003,0.0000,0,0,0,0,1,0,570
1,314430,445442,707468,1.4167,2.2500,1.5882,35861,29157,0.8131,0.1141,...,0.8333,0.5882,0.3704,2,1,0,1,1,0,691
2,161773,517674,2782496,3.2000,17.2000,5.3750,3716,2533,0.6816,0.0230,...,1.0000,0.3125,0.0581,0,1,1,1,1,1,520
3,16014,97685,20818,6.1000,1.3000,0.2131,637,187,0.2936,0.0398,...,1.0000,0.1639,0.7692,0,1,0,0,1,0,654
4,193225,1410542,2589215,7.3000,13.4000,1.8356,5276,2325,0.4407,0.0273,...,1.0000,0.1370,0.0746,0,1,1,0,1,1,552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,20007,22508,457665,1.1250,22.8752,20.3334,1451,861,0.5934,0.0725,...,1.2500,1.1111,0.0546,0,1,0,0,1,0,423
486,20846,268679,23162,12.8888,1.1111,0.0862,1302,533,0.4094,0.0625,...,1.1111,0.0862,1.0000,1,1,0,0,1,0,584
487,89235,1106514,160623,12.4000,1.8000,0.1452,2413,835,0.3460,0.0270,...,1.0000,0.0806,0.5556,0,1,0,0,1,0,631
488,197073,1021195,1863233,5.1818,9.4545,1.8246,8416,3948,0.4691,0.0427,...,0.9091,0.1754,0.0962,2,1,0,1,1,1,572


In [23]:
X_train_transformed, X_val_transformed = cox_box_transform(X_train, X_val)

X_train_transformed

array([[-1.6912027 , -1.81677725, -2.15306889, ...,  0.06401844,
        -0.46137976, -0.40263693],
       [ 1.42847039,  0.44647737,  0.42144924, ...,  0.06401844,
        -0.46137976,  1.99206452],
       [ 0.61999615,  0.57745841,  1.43912406, ...,  0.06401844,
         2.16741192, -1.1304322 ],
       ...,
       [ 0.04224988,  1.34006758, -0.36789948, ...,  0.06401844,
        -0.46137976,  0.68493058],
       [ 0.84043169,  1.25088678,  1.10658551, ...,  0.06401844,
         2.16741192, -0.37057318],
       [ 0.4205704 , -0.66018   ,  0.39670158, ...,  0.06401844,
        -0.46137976,  0.28433732]])

In [24]:
model = RandomForestClassifier( random_state=42)

f1= create_tally(X_train, X_val, y_train, y_val, model)
f1

0.23529411764705882